### Website traffic analysis

In [1]:
#import wget

#wget.download("https://gist.githubusercontent.com/escape-velocity-labs/17c3ba12aef542afe5055a859e0fbd98/raw/af9e2431fb69c76618aeec9940cd5fd6deab8b1f/style.css"
#              , out="assets")
#wget.download("https://gist.githubusercontent.com/escape-velocity-labs/81b8225d72306541018bf868f3a23436/raw/c14fdac538abc45f8e19c4828bb015aa931c41c5/web_data.csv"
#              , out="data")

In [2]:
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [3]:
app = JupyterDash(__name__)
app.title = "Website analytics"

c:\Users\Vizsy.Istvan\Development\Data_analyst_practices\venv\Lib\site-packages\dash\dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [4]:
sitedata = pd.read_csv("./data/web_data.csv")
sitedata.head(7)

,date,time,weekday,os,country,page
0,2021-10-01,00:00:00,Friday,Windows,United States,Detail
1,2021-10-01,00:00:00,Friday,Mac,United Kingdom,Shopping Cart
2,2021-10-01,00:00:00,Friday,Linux,Brazil,Detail
3,2021-10-01,00:00:00,Friday,Windows,Canada,Detail
4,2021-10-01,00:00:00,Friday,Linux,Germany,Landing
5,2021-10-01,00:00:00,Friday,Mac,New Zealand,Landing
6,2021-10-01,00:00:00,Friday,Mac,Russia,Shopping Cart


In [5]:
colors = ["#f2fffb", "#98ffe0", "#6df0c8", "#59dab2",
          "#31c194", "#25a27b", "#188463", "#11684d"]

custom_theme = pio.templates["plotly_dark"]
custom_theme.layout.update({
    "paper_bgcolor": "#1f2630",
    "plot_bgcolor": "#1f2630",
    "colorway": colors,
    "colorscale_sequential": colors,
    "font": {
        "color": "#2cfec1"
    },
    "margin": {
        "t": 75,
        "r": 50,
        "b": 100,
        "l": 75
    }
})

pio.templates.default = custom_theme

### Dashboard structure

#### Dashboard header

In [6]:
header = html.Div(
    id="header",
    children=[
        html.H4("Website traffic analytics"),
        html.P(
            id="description",
            children="This application will display the evolution and key variables related to an e-commerce website."
        )
    ]
)

#### Evolution of website visits

In [7]:
visits_per_day = sitedata.value_counts("date", sort=False)
visits_per_day.head()

date
2021-10-01    307
2021-10-02    308
2021-10-03    342
2021-10-04    325
2021-10-05    331
Name: count, dtype: int64

In [8]:
type(visits_per_day)

pandas.core.series.Series

In [9]:
trend_title = html.P(className="chart-header", children="Website visits per day")

In [10]:
labels = {"date": "Date", "value": "Value"}
trend_figure = px.line(visits_per_day, labels=labels, markers=True)
trend_figure.layout.update(showlegend=False)
#trend_figure.show()

Layout({
    'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
    'showlegend': False,
    'template': '...',
    'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Date'}},
    'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Value'}}
})

In [11]:
trend_graph = dcc.Graph(figure=trend_figure, className="graph")

#### First row

In [12]:
first_row = html.Div(
    className="graph-row",
    children=[
        html.Div(
            className="graph-container",
            children=[
                trend_title,
                trend_graph
            ]
        )
    ]
)

#### Sales funnel

In [13]:
funnel_title = html.P(className="chart-header", children="Sales funnel")

In [14]:
@app.callback(Output("funnel-graph", "figure"),
              [Input("os-dropdown", "value"), Input("country-dropdown", "value")])
def update_funnel_graph(os, country):
    subset = sitedata
    if os:
        subset = subset.query(f"os == '{os}'")
    if country:
        subset = subset.query(f"country == '{country}'")

    visits_per_day = subset.value_counts("page")
    fig = px.funnel(visits_per_day, color=visits_per_day)
    
    return fig

In [15]:
os_dropdown = dcc.Dropdown(
    id="os-dropdown",
    placeholder="Filter by oprating system",
    options=[
        {"label": "Windows", "value": "Windows"},
        {"label": "Android", "value": "Android"},
        {"label": "Mac", "value": "Mac"},
        {"label": "iOS", "value": "iOS"},
        {"label": "Linux", "value": "Linux"}
    ]
)

country_dropdown = dcc.Dropdown(
    id="country-dropdown",
    placeholder="Filter by country",
    options=[
        {"label": "United States", "value": "United States"},
        {"label": "Canada", "value": "Canada"},
        {"label": "United Kingdom", "value": "United Kingdom"},
        {"label": "Australia", "value": "Australia"},
        {"label": "New Zealand", "value": "New Zealand"},
        {"label": "Brazil", "value": "Brazil"},
        {"label": "Russia", "value": "Russia"},
        {"label": "Germany", "value": "Germany"},
        {"label": "Mexico", "value": "Mexico"}
    ]
)

In [16]:
funnel_controls = html.Div(
    className="controler-row",
    children=[
        os_dropdown,
        country_dropdown
            ]
)

In [17]:
funnel_graph = dcc.Graph(id="funnel-graph", className="graph")

#### Distribution of visits by attribute

In [18]:
pie_title = html.P(className="chart-header", children="Visit distribution")

In [19]:
@app.callback(Output("pie-graph", "figure"), [Input("pie-radio", "value")])
def update_pie_chart(data):
    fig = px.pie(sitedata, data)
    return fig

In [20]:
pie_radio = dcc.RadioItems(
    id="pie-radio",
    options=[
        {"label": "Operating System", "value": "os"},
        {"label": "Country", "value": "country"},
    ],
    value="os"
)

In [21]:
pie_controls = html.Div(
    className="controler-row",
    children=[
        pie_radio
            ]
)

In [22]:
pie_graph = dcc.Graph(id="pie-graph") #, className="graph"

#### Second row

In [23]:
second_row = html.Div(
    className="graph-row",
    children=[
        html.Div(
            className="graph-container",
            children=[
                funnel_title,
                funnel_controls,
                funnel_graph
            ]
        ),
        html.Div(
            className="graph-container",
            children=[
                pie_title,
                pie_controls,
                pie_graph
            ]
        )
    ]
)

#### Visits by date and time

In [24]:
heatmap_title = html.P(className="chart-header", children="Key activity periods")

In [25]:
visits_day_time = sitedata.groupby(["weekday", "time"]).size().reset_index(name="counts")
visits_day_time.head()
#visits_per_day.shape
#type(visits_per_day)


,weekday,time,counts
0,Friday,00:00:00,44
1,Friday,01:00:00,48
2,Friday,02:00:00,53
3,Friday,03:00:00,45
4,Friday,04:00:00,55


In [26]:
visits_day_time_piv = visits_day_time.pivot_table(values="counts", index="time", columns="weekday")
visits_day_time_piv.head()

weekday,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
time,,,,,,,
00:00:00,44.0,38.0,42.0,39.0,47.0,41.0,43.0
01:00:00,48.0,46.0,53.0,48.0,46.0,53.0,45.0
02:00:00,53.0,44.0,58.0,46.0,51.0,44.0,37.0
03:00:00,45.0,35.0,50.0,54.0,35.0,39.0,66.0
04:00:00,55.0,46.0,43.0,51.0,41.0,45.0,46.0


In [27]:
heatmap_figure = go.Figure(
    data=go.Heatmap(
        x=visits_day_time_piv.columns.tolist(),
        y=visits_day_time_piv.index.tolist(),
        z=visits_day_time_piv.values.tolist(),
        autocolorscale=True,
        xgap=4,
        ygap=1
    )
)
layout = heatmap_figure.layout.yaxis.update(autorange="reversed",
                                            tickvals=[0, 6, 12, 18], ticktext=["0 AM", "6 AM", "12 AM", "6 PM"])

In [28]:
heatmap_graph = dcc.Graph(figure=heatmap_figure, className="graph")

#### Distribution by country

In [29]:
country_title = html.P(className="chart-header", children="Visits by country")

In [30]:
country_figure = px.histogram(sitedata, x="country", color="country")
layout = country_figure.layout.update(showlegend=False)

In [31]:
country_graph = dcc.Graph(figure=country_figure, className="graph")

#### Third row

In [32]:
third_row = html.Div(
    className="graph-row",
    children=[
        html.Div(
            className="graph-container",
            children=[
                heatmap_title,
                heatmap_graph
            ]
        ),
        html.Div(
            className="graph-container",
            children=[
                country_title,
                country_graph
            ]
        )
    ]
)

#### Create the dashboard

In [33]:
app.layout = html.Div(
    id="root",
    children=[
        header,
        first_row,
        second_row,
        third_row
    ]
)

In [34]:
app.run_server(debug=True, mode="external", port=8049) #mode="inline"

Dash app running on http://127.0.0.1:8049/
